<a href="https://colab.research.google.com/github/Gregrs400/cmpsc472Project2/blob/main/cmpsc472Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%writefile cmpsc472Project2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>

int main() {
  // Create array to represent the landscape
  int landscape[20][20];
  // Fill array with 0's
  for (int i = 0; i < 20; i++) {
    for (int j = 0; j < 20; j++) {
      landscape[i][j] = 0;
    }
  }
  printf("array value at position 12,15: %d", landscape[12][15]);
  return 0;
}

Overwriting cmpsc472Project2.c


In [8]:
%%shell
gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2

array value at position 12,15: 0